# Evaluate Retriever

This playground connect to Chroma Vector Data Base build wiht 5 documents:
- [COCACOLA_2021_10K.pdf](../../data/financebench/COCACOLA_2021_10K.pdf)
- [PFIZER_2021_10K.pdf](../../data/financebench/PFIZER_2021_10K.pdf)
- [VERIZON_2022_10K.pdf](../../data/financebench/VERIZON_2022_10K.pdf)
- [PEPSICO_2021_10K.pdf](../../data/financebench/PEPSICO_2022_10K.pdf)
- [NETFLIX_2017_10K.pdf](../../data/financebench/NETFLIX_2017_10K.pdf)

In [1]:
import pandas as pd
import os
from langchain.vectorstores import Chroma
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.append(os.path.abspath('../../src'))
from azure_openai_conn import OpenAIembeddings, llm

In [2]:
embeddings = OpenAIembeddings()
query = "What is the Coca Cola Balance Sheet?"
vectordb = Chroma(persist_directory="db_chroma", embedding_function=embeddings)
docs = vectordb.similarity_search(query)
print(docs[0].page_content)

THE COCA-COLA COMPANY AND SUBSIDIARIES
CONSOLIDATED BALANCE SHEETS
(In millions except par value)
December 31, 2021 2020
ASSETS
Current Assets   
Cash and cash equivalents $ 9,684 $ 6,795 
Short-term investments 1,242 1,771 
Total Cash, Cash Equivalents and Short-Term Investments 10,926 8,566 
Marketable securities 1,699 2,348 
Trade accounts receivable, less allowances of $516 and $526, respectively 3,512 3,144 
Inventories 3,414 3,266 
Prepaid expenses and other current assets 2,994 1,916 
Total Current Assets 22,545 19,240 
Equity method investments 17,598 19,273 
Other investments 818 812 
Other noncurrent assets 6,731 6,184 
Deferred income tax assets 2,129 2,460 
Property, plant and equipment — net 9,920 10,777 
Trademarks with indefinite lives 14,465 10,395 
Goodwill 19,363 17,506 
Other intangible assets 785 649 
Total Assets $ 94,354 $ 87,296 
LIABILITIES AND EQUITY
Current Liabilities   
Accounts payable and accrued expenses $ 14,619 $ 11,145 
Loans and notes payable 3,307 2,

In [3]:
financebenchQA = pd.read_csv('../../data/financebench_sample_150.csv')

In [4]:
financebenchQA

,financebench_id,doc_name,doc_link,doc_period,question_type,question,answer,evidence_text,page_number
0,financebench_id_03029,3M_2018_10K,https://investors.3m.com/financials/sec-filing...,2018,metrics-generated,What is the FY2018 capital expenditure amount ...,$1577.00,Table of Contents \n3M Company and Subsidiarie...,60
1,financebench_id_04672,3M_2018_10K,https://investors.3m.com/financials/sec-filing...,2018,metrics-generated,Assume that you are a public equities analyst....,$8.70,Table of Contents \n3M Company and Subsidiarie...,58
2,financebench_id_00499,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,domain-relevant,Is 3M a capital-intensive business based on FY...,"No, the company is managing its CAPEX and Fixe...",3M Company and Subsidiaries\n Consolidated Sta...,"48,50,52"
3,financebench_id_01226,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,domain-relevant,What drove operating margin change as of FY202...,Operating Margin for 3M in FY2022 has decrease...,"SG&A, measured as a percent of sales, increase...",27
4,financebench_id_01865,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,novel-generated,"If we exclude the impact of M&A, which segment...",The consumer segment shrunk by 0.9% organically.,Worldwide Sales Change\nBy Business Segment Or...,25
...,...,...,...,...,...,...,...,...,...
145,financebench_id_00215,VERIZON_2022_10K,https://www.verizon.com/about/sites/default/fi...,2022,domain-relevant,Is Verizon a capital intensive business based ...,Yes. Verizon's capital intensity ratio was app...,Consolidated Balance Sheets \nVerizon Communic...,"56, 23"
146,financebench_id_00566,VERIZON_2022_10K,https://www.verizon.com/about/sites/default/fi...,2022,domain-relevant,Has Verizon increased its debt on balance shee...,No. Verizon's debt decreased by $229 million.,"At December 31, Maturities \nInterest \nRates ...",77
147,financebench_id_06247,WALMART_2018_10K,https://d18rn0p25nwr6d.cloudfront.net/CIK-0000...,2018,metrics-generated,What is FY2018 days payable outstanding (DPO) ...,42.69,Walmart Inc.\nConsolidated Statements of Incom...,"57,59"
148,financebench_id_04784,WALMART_2019_10K,https://d18rn0p25nwr6d.cloudfront.net/CIK-0000...,2019,metrics-generated,Based on the information provided primarily in...,0.2%,Walmart�Inc.\nConsolidated�Statements�of�Incom...,48


# Truelens Triad RAG Evaluation

In [3]:
from langchain.chains import RetrievalQA

# completion llm
llm = AzureChatOpenAI(model_name="gtp35turbo-latest")

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

In [4]:
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')
AZURE_OPENAI_API_KEY =os.getenv('OPENAI_API_KEY')

In [5]:
from trulens_eval.feedback.provider.openai import AzureOpenAI
openai = AzureOpenAI(deployment_name="gtp35turbo-latest")


OpenAIError: Missing credentials. Please pass one of `api_key`, `azure_ad_token`, `azure_ad_token_provider`, or the `AZURE_OPENAI_API_KEY` or `AZURE_OPENAI_AD_TOKEN` environment variables.

In [8]:
# Imports main tools for eval
from trulens_eval import TruChain, Feedback, Tru, feedback, Select
import numpy as np
tru = Tru()
# OpenAI as feedback provider
openai = feedback.OpenAI()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [10]:
# Question/answer relevance between overall question and answer.
qa_relevance = Feedback(openai.relevance).on_input_output()

✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [11]:
# Question/statement relevance between question and each context chunk.
qs_relevance = Feedback(openai.qs_relevance).on_input().on(Select.Record.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content).aggregate(np.mean)


✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to __record__.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content .


In [ ]:
# wrap with TruLens
truchain = TruChain(qa,
    app_id='Chain1_WikipediaQA',
    feedbacks=[qa_relevance, qs_relevance])

truchain("What is the Coca Cola Balance Sheet?")


`call` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_` method:
        ```python
        app # your app
        tru_app_recorder: TruChain = TruChain(app, ...)
        result = tru_app_recorder.with_(app, ...args/kwargs-to-app...)
        ```
    (2) Use TruChain as a context manager: 
        ```python
        app # your app
        tru_app_recorder: TruChain = TruChain(app, ...)
        with tru_app_recorder:
            result = app(...args/kwargs-to-app...)
        
        ```



{'query': 'What is the Coca Cola Balance Sheet?',
 'result': "The Coca-Cola Company's balance sheet is a financial statement that provides a snapshot of the company's assets, liabilities, and shareholders' equity at a specific point in time. It shows the financial position of the company as of December 31, 2021 and 2020. Here is a summary of the key components:\n\nAssets:\n- Current Assets: Includes cash and cash equivalents, short-term investments, marketable securities, trade accounts receivable, inventories, and prepaid expenses.\n- Equity Method Investments: Represents the company's investments in other companies in which it has significant influence.\n- Other Investments: Includes investments in other companies.\n- Other Noncurrent Assets: Represents noncurrent assets other than property, plant, and equipment, trademarks, and goodwill.\n- Deferred Income Tax Assets: Represents tax assets that will be realized in future periods.\n- Property, Plant, and Equipment: Represents the com

openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: b82effcf********************311c. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=3.
openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: b82effcf********************311c. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=3.
openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: b82effcf********************311c. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries re

# Other Evaluations

In [5]:
qa = financebenchQA[['question_type', 'question', 'answer']]

In [6]:
# Iterate through each row and print the question and answer
def split_in_chunks(answer, chunk_size=100):
    return [answer[i:i+chunk_size] for i in range(0, len(answer), chunk_size)]


for index, row in qa.iterrows():
    
    print(f"Q-{index}: {row['question_type']}")
    
    for chunk in split_in_chunks(row['question']):
        print(f"Q: {chunk}")
    print('-'*5)
    for chunk in split_in_chunks(row['answer']):
        print(f"A: {chunk}")
    print('-'*100)

Q-0: metrics-generated
Q: What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the quest
Q: ion by relying on the details shown in the cash flow statement.
-----
A: $1577.00
----------------------------------------------------------------------------------------------------
Q-1: metrics-generated
Q: Assume that you are a public equities analyst. Answer the following question by primarily using info
Q: rmation that is shown in the balance sheet: what is the year end FY2018 net PPNE for 3M? Answer in U
Q: SD billions.
-----
A: $8.70
----------------------------------------------------------------------------------------------------
Q-2: domain-relevant
Q: Is 3M a capital-intensive business based on FY2022 data?
-----
A: No, the company is managing its CAPEX and Fixed Assets pretty efficiently, which is evident from bel
A: ow key metrics:
CAPEX/Revenue Ratio: 5.1%
Fixed assets/Total Assets: 20%
Return on Assets= 12.4%
---------------------------------

In [7]:
q = 'What is the FY2022 capital expenditure amount (in USD millions) for Verizon? Give a response to the question by relying on the details shown in the cash flow statement.'

In [8]:
vectordb = Chroma(persist_directory="db_chroma", embedding_function=embeddings)
docs = vectordb.similarity_search(q)
print(docs[0].page_content)

(dollars in millions) 
Years Ended December 31, 2022 2021 
Net cash provided by operating activities $ 37,141 $ 39,539 
Less Capital expenditures (including capitalized software) 23,087 20,286 
Free cash flow $ 14,054 $ 19,253   
The decrease in free cash flow during 2022 is a reflection of the decrease in operating cash flows, as well as the increase in 
capital expenditures, both of which are discussed above.
Verizon 2022 Annual Report on Form 10-K                                42


In [10]:
q= 'We want to calculate a financial metric. Please help us compute it by basing your answers off of the statement of income and the statement of cash flows. Here is the question: what is the FY2015 unadjusted EBITDA % margin for Netflix? Calculate unadjusted EBITDA using unadjusted operating income and D&A (from cash flow statement).'

In [12]:
vectordb = Chroma(persist_directory="db_chroma", embedding_function=embeddings)
docs = vectordb.similarity_search(q)
print(docs[0].page_content), print(docs[0].metadata)

Consolidated EBITDA is calculated by adding back interest, taxes, depreciation and amortization expense to net income. 
Consolidated Adjusted EBITDA is calculated by excluding from Consolidated EBITDA the effect of the following non-operational 
items: equity in earnings of unconsolidated businesses and other income and expense, net, as well as the effect of certain 
special items. We believe that this measure is useful to management, investors and other users of our financial information in 
evaluating the effectiveness of our operations and underlying business trends in a manner that is consistent with management’s 
evaluation of business performance. We believe that Consolidated Adjusted EBITDA is widely used by investors to compare a 
company’s operating performance to its competitors by minimizing impacts caused by differences in capital structure, taxes, and 
depreciation and amortization policies. Further, the exclusion of non-operational items and special items enables comparab

(None, None)

Fail to retrieve the right source.

In [23]:
docs = vectordb.similarity_search(
    q,
    k=3,
    filter={"source":"../../data/financebench/NETFLIX_2017_10K.pdf"}
)

In [24]:
for d in docs:
    print(d.metadata)

{'page': 43, 'source': '../../data/financebench/NETFLIX_2017_10K.pdf', 'start_index': 0}
{'page': 27, 'source': '../../data/financebench/NETFLIX_2017_10K.pdf', 'start_index': 4367}
{'page': 41, 'source': '../../data/financebench/NETFLIX_2017_10K.pdf', 'start_index': 0}


In [26]:
docs[1].page_content

'e, operating income, cash flow used in operating activities, or any other measure of financial performance or liquidity presented inaccordance with GAAP.\nIn assessing liquidity in re\nlation to our results of operations, we compare free cash flow to net income, noting that the three major recurring differences areexcess content payments ove\nr amortization, non-cash stock-based compensation expense and other working capital differences. Working capital differencesinclude deferred revenue, ta\nxes and semi-annual interest payments on our outstanding debt. Our receivables from members generally settle quickly and deferredrevenue is a source of cash \nflow.26'